In [1]:
import magcolloids as mgc
import os
import sys

sys.path.insert(0, '../icenumerics')


import icenumerics as ice
import matplotlib.pyplot as plt

# From Lammpstrj to Vertex Dataframe + Non inferred vertices

In this script, I will convert the lammpstrj data to a vertex dataframe. In addition, I will try if this process works by adding a non inferred topology of the lattice. 

In [17]:
data_directory = "/home/carolina/CairoLattice_ACI_PROJECT/Scripts/Output_Script-10.0"

In [18]:
load_obj = mgc.trj_lazyread(os.path.join(data_directory,
                                         "first_correct_test.lammpstrj"),
                                            output = ["x","y","z","mux","muy","muz"])    
bnd = load_obj.get_bounds()
trj = load_obj.read_trj()
col_trj = ice.get_ice_trj(trj,bnd, atom_types = 1, trap_types = [2,3])

ValueError: cannot handle a non-unique multi-index!

the function ice.get_ice_trj() in order to extract the col_trj doesn't work. Until I find a way to fix it I will infer the vertices values by using the col object. 

In [ ]:
col_trj

In [5]:
bnd

,x_min,x_max,y_min,y_max,z_min,z_max
frame,,,,,,
0,-28.499315,1107.120815,-85.268868,1050.351263,-0.02,0.02
100,-28.499315,1107.120815,-85.268868,1050.351263,-0.02,0.02
200,-28.499315,1107.120815,-85.268868,1050.351263,-0.02,0.02
300,-28.886305,1107.426989,-85.748100,1050.836718,-0.02,0.02
400,-28.752037,1107.777939,-85.621523,1050.728483,-0.02,0.02
...,...,...,...,...,...,...
199600,-28.876472,1107.477494,-85.712629,1050.928775,-0.02,0.02
199700,-28.876472,1107.477494,-85.712629,1050.928775,-0.02,0.02
199800,-28.941949,1107.673468,-85.536897,1050.644868,-0.02,0.02


In [19]:
def unwrap_trj(trj,bounds):
    """ Unwraps trj around periodic boundaries"""
    trj2 = trj.copy(deep=True)

    def unwrap(p):
        p.iloc[:] = np.unwrap(p,axis=0)
        return p

    for c in trj.columns:
        trj2[c] = (trj2[c] - bounds[c+"_min"].values)/(bounds[c+"_max"].values - bounds[c+"_min"].values)

    trj2 = (trj2*2*np.pi).groupby("id").apply(unwrap)/(2*np.pi)

    for c in trj.columns:
        trj2[c] = trj2[c]*(bounds[c+"_max"].values - bounds[c+"_min"].values) + bounds[c+"_min"].values

    return trj2

In [20]:
import numpy as np
import pandas as pd
import scipy.spatial as spa


In [21]:
trap_types = [2,3]
atom_types = 1
if trap_types is None:
    trap_types = [2]

try: 
    traps = trj[trj.type.isin(trap_types)].copy(deep=True)
except TypeError: 
    traps = trj[trj.type.isin([trap_types])].copy(deep=True)

try:
    atoms = trj[trj.type.isin(atom_types)].copy(deep=True)
except TypeError:
    atoms = trj[trj.type.isin([atom_types])].copy(deep=True)

traps = traps.rename(columns = {"mux":"dx","muy":"dy","muz":"dz"})
atoms = unwrap_trj(atoms.filter(["x","y","z"]),bnd.loc[[0]])

trj = []

In [22]:
atoms

x           y    z
frame  id                                 
0      1    -3.552714e-15  -19.192400  0.0
       2    -9.192390e+00   28.384800  0.0
       3     1.919240e+01   28.384800  0.0
       4    -9.192390e+00  -28.384800  0.0
       5     1.919240e+01  -28.384800  0.0
...                   ...         ...  ...
200000 1996  1.078590e+03  965.150024  0.0
       1997  1.067930e+03  993.499023  0.0
       1998  1.097820e+03  993.370972  0.0
       1999  1.069420e+03  936.611023  0.0
       2000  1.098040e+03  936.570984  0.0

[4002000 rows x 3 columns]

In [23]:
## It turns out that the traps are not in the same order as the particles, when the system is bidisperse. 
# we first reindex the traps so that they start at zero, and increase consecutively
traps_id = traps.index.get_level_values("id").unique()
reindex_traps = pd.Series({t:i for i, t in enumerate(traps_id)})
traps.reset_index(inplace = True)
traps.id = traps.id.map(reindex_traps)
traps = traps.set_index(["frame","id"]).sort_index()

In [24]:
traps

type            x           y    z       dx       dy   dz
frame  id                                                             
0      0        2     0.000000    0.000000  0.0  -0.0000 -38.3848 -0.0
       1        2    56.769600    0.000000  0.0 -38.3848  -0.0000 -0.0
       2        2     0.000000  -56.769600  0.0  38.3848   0.0000  0.0
       3        2    56.769600  -56.769600  0.0   0.0000  38.3848  0.0
       4        2   113.539001    0.000000  0.0  -0.0000 -38.3848 -0.0
...           ...          ...         ...  ...      ...      ...  ...
200000 1995     3  1050.239990  950.890015  0.0   0.0000  10.0000  0.0
       1996     3  1064.430054  993.466980  0.0  10.0000   0.0000  0.0
       1997     3  1092.810059  993.466980  0.0  10.0000   0.0000  0.0
       1998     3  1064.430054  936.697998  0.0  10.0000   0.0000  0.0
       1999     3  1092.810059  936.697998  0.0  10.0000   0.0000  0.0

[4002000 rows x 7 columns]

In [39]:
# we calculate the distance between traps and particles in the first frame,
# and we build an atom index from the minimization of this distance
distances = spa.distance.cdist(traps.loc[200000,["x","y","z"]], atoms.loc[200000,["x","y","z"]])
reindex_atoms = pd.Series({a+1:t for a, t in enumerate(np.argmin(distances, axis = 0))})

In [40]:
distances

array([[  29.64811433,   30.00201514,    1.7885693 , ..., 1480.53859181,
        1421.58342657, 1443.21070369],
       [  71.82234735,   55.52446791,   56.96149969, ..., 1438.94812661,
        1379.38433009, 1400.50324359],
       [  85.4276367 ,   85.82941973,   54.98881456, ..., 1519.2115898 ,
        1459.61100718, 1480.68141165],
       ...,
       [1465.09335281, 1450.97404662, 1478.2120962 , ...,    5.01080755,
          61.4792049 ,   57.13586498],
       [1406.54554084, 1392.226033  , 1419.19113925, ...,   65.77773838,
           4.99074816,   33.61022535],
       [1428.3280715 , 1413.85852503, 1440.60240388, ...,   56.8939796 ,
          23.39017636,    5.23152256]])

In [41]:
# now we reindex the atoms
atoms.reset_index(inplace = True)
atoms.id = atoms.id.map(reindex_atoms)
atoms = atoms.set_index(["frame","id"]).sort_index()

In [42]:
## create a relative position vector. This goes from the center of the trap to the position of the particle
colloids = atoms-traps
colloids = colloids[["x","y","z"]]
colloids.columns = ["cx","cy","cz"]

In [43]:
len(atoms)

4002000

In [44]:
len(traps)

4002000

In [45]:
traps

type            x           y    z       dx       dy   dz
frame  id                                                             
0      0        2     0.000000    0.000000  0.0  -0.0000 -38.3848 -0.0
       1        2    56.769600    0.000000  0.0 -38.3848  -0.0000 -0.0
       2        2     0.000000  -56.769600  0.0  38.3848   0.0000  0.0
       3        2    56.769600  -56.769600  0.0   0.0000  38.3848  0.0
       4        2   113.539001    0.000000  0.0  -0.0000 -38.3848 -0.0
...           ...          ...         ...  ...      ...      ...  ...
200000 1995     3  1050.239990  950.890015  0.0   0.0000  10.0000  0.0
       1996     3  1064.430054  993.466980  0.0  10.0000   0.0000  0.0
       1997     3  1092.810059  993.466980  0.0  10.0000   0.0000  0.0
       1998     3  1064.430054  936.697998  0.0  10.0000   0.0000  0.0
       1999     3  1092.810059  936.697998  0.0  10.0000   0.0000  0.0

[4002000 rows x 7 columns]

In [46]:
colloids.head(15)

cx  cy  cz
frame id            
0     0  NaN NaN NaN
      1  NaN NaN NaN
      2  NaN NaN NaN
      3  NaN NaN NaN
      4  NaN NaN NaN
      5  NaN NaN NaN
      6  NaN NaN NaN
      7  NaN NaN NaN
      8  NaN NaN NaN
      9  NaN NaN NaN
      10 NaN NaN NaN
      11 NaN NaN NaN
      12 NaN NaN NaN
      13 NaN NaN NaN
      14 NaN NaN NaN

In [47]:
traps = pd.concat([traps,colloids],axis=1)
colloids = []
atoms = []

ValueError: cannot handle a non-unique multi-index!

In [ ]:
## Flip those traps that are not pointing in the  direction of the colloids
flip = np.sign((traps[["dx","dy","dz"]].values*traps[["cx","cy","cz"]].values).sum(axis=1))
traps[["dx","dy","dz"]] = traps[["dx","dy","dz"]].values*flip[:,np.newaxis]

## make the direction vector unitary
mag = np.sign((traps[["dx","dy","dz"]].values**2).sum(axis=1))
traps[["dx","dy","dz"]] = traps[["dx","dy","dz"]].values*mag[:,np.newaxis]

After checking the function carefully I realize that the reindexing is not working correctly due to it match the particles and the traps by distances. In the shakti lattice, sometimes and in particular for the particles located at the center of the plaquette, some particles are nearer the trap next to it than the belonging trap. 